# MovieLens Data: Initial Data Analysis

In this notebook we will explore the basic properties of MovieLens datasets.

The datasets we are going to work with is a development dataset taken from MovieLens movie recommendation service. It contains 100836 movie ratings and free text tagging by 610 users. The rating and tagging were performed between March 29, 1996 up to September, 24, 2018.

In this section we will present the data describing the datasets, and the links between the datasets. We will also comput a basic statistics for the MovieLens datasets. The data provided in this section will be useful for further analysis.

We will be using Spark for the exploratory data analysis.

### 1. Datasets Description

The MovieLens dataset contains the following files:

- links.csv
- movies.csv 
- ratings.csv
- tag.csv

Let's explore what is in the each file.

In [62]:
# Project Dependencies
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

# Setting a Spark Session
spark = SparkSession.builder.appName("Movies_With_No_Genre").getOrCreate()


#### movies.csv dataset

In [63]:
# movies.csv file

movies = spark.read.csv(
    path = "./data/movies.csv",
    sep = ",",
    header = True,
    quote = '"',
    schema = "movieID INT, title STRING, genres STRING"
)
movies.printSchema()
movies.show(n=10, truncate = False)

root
 |-- movieID: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)

+-------+----------------------------------+-------------------------------------------+
|movieID|title                             |genres                                     |
+-------+----------------------------------+-------------------------------------------+
|1      |Toy Story (1995)                  |Adventure|Animation|Children|Comedy|Fantasy|
|2      |Jumanji (1995)                    |Adventure|Children|Fantasy                 |
|3      |Grumpier Old Men (1995)           |Comedy|Romance                             |
|4      |Waiting to Exhale (1995)          |Comedy|Drama|Romance                       |
|5      |Father of the Bride Part II (1995)|Comedy                                     |
|6      |Heat (1995)                       |Action|Crime|Thriller                      |
|7      |Sabrina (1995)                    |Comedy|Romance                    

From the results above we can see that the **movies.csv** file contains

1. **movieID** - movie ID;
2. **title** - movie title;
3. **genres** - a pipe-separated list of genres that the movie was assigned.



#### ratings.csv dataset

In [64]:
# ratings.csv file

ratings = (
    spark.read.csv(
        path="./data/ratings.csv",
        sep=",",
        header=True,
        quote='"',
        schema="userId INT, movieId INT, rating DOUBLE, timestamp INT",
    )
    .withColumn("timestamp", f.to_timestamp(f.from_unixtime("timestamp")))
)

ratings.printSchema()
ratings.show(10)

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: timestamp (nullable = true)

+------+-------+------+-------------------+
|userId|movieId|rating|          timestamp|
+------+-------+------+-------------------+
|     1|      1|   4.0|2000-07-30 18:45:03|
|     1|      3|   4.0|2000-07-30 18:20:47|
|     1|      6|   4.0|2000-07-30 18:37:04|
|     1|     47|   5.0|2000-07-30 19:03:35|
|     1|     50|   5.0|2000-07-30 18:48:51|
|     1|     70|   3.0|2000-07-30 18:40:00|
|     1|    101|   5.0|2000-07-30 18:14:28|
|     1|    110|   4.0|2000-07-30 18:36:16|
|     1|    151|   5.0|2000-07-30 19:07:21|
|     1|    157|   5.0|2000-07-30 19:08:20|
+------+-------+------+-------------------+
only showing top 10 rows



From the result above we can see that the **ratings.csv** file contains the following columns

1. **userID** - User ID
2. **movieID** - Movie ID
3. **rating** - Rating
4. **timestamp** - Timestamp

The ratings.csv file contains a movie rating provided by a user on a certain date.

#### tags.csv dataset

In [65]:
# tags.csv file

tags = (
    spark.read.csv(
        path="./data/tags.csv",
        sep=",",
        header=True,
        quote='"',
        schema="userId INT, movieId INT, tag STRING, timestamp INT",
    )
    .withColumn("timestamp", f.to_timestamp(f.from_unixtime("timestamp")))
)

tags.printSchema()
tags.show(10)

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- tag: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)

+------+-------+-----------------+-------------------+
|userId|movieId|              tag|          timestamp|
+------+-------+-----------------+-------------------+
|     2|  60756|            funny|2015-10-24 19:29:54|
|     2|  60756|  Highly quotable|2015-10-24 19:29:56|
|     2|  60756|     will ferrell|2015-10-24 19:29:52|
|     2|  89774|     Boxing story|2015-10-24 19:33:27|
|     2|  89774|              MMA|2015-10-24 19:33:20|
|     2|  89774|        Tom Hardy|2015-10-24 19:33:25|
|     2| 106782|            drugs|2015-10-24 19:30:54|
|     2| 106782|Leonardo DiCaprio|2015-10-24 19:30:51|
|     2| 106782|  Martin Scorsese|2015-10-24 19:30:56|
|     7|  48516|     way too long|2007-01-25 01:08:45|
+------+-------+-----------------+-------------------+
only showing top 10 rows



The file **tags.csv** contains the following fields:

1. **userID** - User ID;
2. **movieID** - Movie ID;
3. **tag** - Movie Tag
4. **timestamp** - the time when tag was assigned

It contains tags assigned by a user to a movie. The user can assign multiple tags to one movie.

#### links.csv dataset

This dataset links the movie in the MovieLens database to the same movie int IMDB and TMDB databases.

In [66]:
# links.csv file

links = spark.read.csv(
    path = "./data/links.csv",
    sep = ",",
    header = True,
    quote = '"',
    schema = "movieID INT, imdbID INT, tmdbID INT"
)
links.printSchema()
links.show(n=10, truncate = False)

root
 |-- movieID: integer (nullable = true)
 |-- imdbID: integer (nullable = true)
 |-- tmdbID: integer (nullable = true)

+-------+------+------+
|movieID|imdbID|tmdbID|
+-------+------+------+
|1      |114709|862   |
|2      |113497|8844  |
|3      |113228|15602 |
|4      |114885|31357 |
|5      |113041|11862 |
|6      |113277|949   |
|7      |114319|11860 |
|8      |112302|45325 |
|9      |114576|9091  |
|10     |113189|710   |
+-------+------+------+
only showing top 10 rows



The **links.csv** file contains the following columns:

1. movieID - Movie ID;
2. imdbID - Movie ID in IMDB database: https://www.imdb.com/
3. tmdbID - Movie ID in TMDB database: https://www.themoviedb.org

### 2. MovieLens Dataset Statistics

In [68]:
print("\n*** MovieLens Dataset Statistics ***\n")

# Number of Movies in the Dataset
nMovies = movies.count()
print("Number of movies in the dataset: {0}".format(nMovies))
 
# Number of Rated Movies
nRated = ratings.select("movieID").distinct().count()
print("Number of rated movies: {0}".format(nRated))

# Number of Tagged Movies
nTagged = tags.select("movieID").distinct().count()
print("Number of tagged movies: {0}".format(nTagged))

# Number of Linked Movies
nLinked = links.select("movieID").distinct().count()
print("Number of movies linked to IMDB or to TMDB: {0}".format(nLinked))

print("")

# Number of Movie Genres
nGenres = (
    (
        movies
            .withColumn("genres_array", f.split("genres", "\|"))
            .withColumn("genre", f.explode("genres_array"))
    )
    .select("genre").distinct()
).count()
print("Number of unique movie genres: {0}".format(nGenres))

print("")

# Number of users
nUsers = ratings.select("userID").distinct().count()
print("Number of unique users: {0}".format(nUsers))

# Number of user ratings
nRatings = ratings.count()
print("The dataset contains {0} user ratings.".format(nRatings))

# Number of user tags
nTags = tags.count()
print("The dataset contains {0} unique tags.".format(nTags))

print("")

# The date of the first and the last ratings
print("The first and the last rating timestamps")
ratings.agg(
    f.min("timestamp").alias("First Rating Timestamp"),
    f.max("timestamp").alias("Last Rating Timestamp")
).show()
print("")

# The date of the first and the last tag
print("The first and the last tag timestamps")
tags.agg(
    f.min("timestamp").alias("Firts Tag Timestamp"),
    f.max("timestamp").alias("Last Tag Timestamp")
).show()


*** MovieLens Dataset Statistics ***

Number of movies in the dataset: 9742
Number of rated movies: 9724
Number of tagged movies: 1572
Number of movies linked to IMDB or to TMDB: 9742

Number of unique movie genres: 20

Number of unique users: 610
The dataset contains 100836 user ratings.
The dataset contains 3683 unique tags.

The first and the last rating timestamps
+----------------------+---------------------+
|First Rating Timestamp|Last Rating Timestamp|
+----------------------+---------------------+
|   1996-03-29 18:36:55|  2018-09-24 14:27:30|
+----------------------+---------------------+


The first and the last tag timestamps
+-------------------+-------------------+
|Firts Tag Timestamp| Last Tag Timestamp|
+-------------------+-------------------+
|2006-01-13 19:09:12|2018-09-16 11:50:03|
+-------------------+-------------------+

